<div class="alert alert-info">

**Note:**    WARNING: This notebook uses a new, early release Chameleon python API. The notebook and API will likely be updated without notice. To request a new python feature, please create a help desk ticket. 
    

</div>

## Stitching Networks between Chameleon and an ExoGENI Stitchport

An example that can be used to stitch to an ExoGENI Stitchport

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)
- [Get Network by Name](../modules-python/network/get_network_by_name.ipynb)

TODO: add the rest of the module links when they are ready

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [ ]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project
chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name

#Set the region
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

#GENI Pem file
geni_pem_file='/home/pruth/work/geni-pruth.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
stitched_network_name = network_name+"Stitched"
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name_network = username+'LeaseNet'
lease_name_servers = username+'LeaseServers'
lease_name_stitch = username+'LeaseStitch'


#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=2

#Network Config
cidr='192.168.42.0/24'

#optionally set the OpenFlow controller (set ot None for non-OpenFlow swtich)
#of_controller_ip=None
of_controller_ip="192.5.87.215"
of_controller_port="6653"


#Set the name of the VFC
vswitch_name="pruthSDN"

#Stitchport URL
stitchport_url=<your_stitchport_url>
stitchport_vlan=<your_stitchport_vlan>

## Create the Network (OpenFlow Optional)

#### Create a Lease at Chicago

In [ ]:


# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)


# Build list of reservations (in this case there is only one reservation)
reservation_list = []

if of_controller_ip:
    print("OpenFlow")
    add_network_reservation(reservation_list, 
                            network_name=network_name, 
                            physical_network='physnet1',
                            of_controller_ip=of_controller_ip, 
                            of_controller_port=of_controller_port, 
                            vswitch_name=vswitch_name)
else:
    print("No OpenFlow")
    add_network_reservation(reservation_list, 
                            network_name=network_name, 
                            physical_network='physnet1',
                            vswitch_name=vswitch_name)
                           

# Create the lease
chi.blazar().lease.create(name=lease_name_network, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
network_lease = get_lease_by_name(lease_name_network)
    
#Print the lease info
print(json.dumps(network_lease, indent=2))

#### Get the Nework Reservation

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
network_lease = get_lease_by_name(lease_name_network)

network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', network_lease['reservations']))[0]['id']

print("network_reservation_id: " + network_reservation_id)

In [ ]:
#Get the network
network = get_network_by_name(network_name)
#print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
network_vlan = network['provider:segmentation_id']
print('network_vlan: ' + str(network_vlan))

#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [ ]:
subnet = add_subnet(subnet_name, network_name, cidr=cidr)

print(json.dumps(subnet, indent=2))

#### Add a Router

TODO: add links here

In [ ]:
router = create_router(router_name, gw_network_name='public')

print(json.dumps(router, indent=2))

#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

#### Add a Circuit Stitched to ExoGENI

In [ ]:
#Set the region
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []

add_network_reservation(reservation_list,
                        network_name=stitched_network_name, 
                        physical_network='exogeni',
                        vswitch_name=vswitch_name)
                           

# Create the lease
chi.blazar().lease.create(name=lease_name_stitch, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
stitched_network_lease = get_lease_by_name(lease_name_stitch)
    
#Print the lease info
print(json.dumps(stitched_network_lease, indent=2))

#### Get the Stitched Network Reservation and VLAN

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
stitch_lease = get_lease_by_name(lease_name_stitch)

stitch_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', stitch_lease['reservations']))[0]['id']

print("stitch_reservation_id: " + stitch_reservation_id)

chi.set('region_name', 'CHI@UC')  
#Get the network
network = get_network_by_name(stitched_network_name)
#print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
exogeni_vlan = network['provider:segmentation_id']
print('exogeni_vlan: ' + str(exogeni_vlan))



## Create the Servers

In [ ]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name_servers, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
server_lease = get_lease_by_name(lease_name_servers)
    
#Print the lease info
print(json.dumps(server_lease, indent=2))

In [ ]:
#Get the lease by name
server_lease = get_lease_by_name(lease_name_servers)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', server_lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', server_lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

#### Start the Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
#create the server
server = create_server(server_name, 
                       reservation_id=compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit


In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

#### Check the Status of the Circuit

In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

### Delete Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
delete_server_by_name(server_name)

#### De-configure Network
TODO: break up into steps

In [ ]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

In [ ]:
delete_router_by_name(router_name)

In [ ]:
delete_subnet_by_name(subnet_name)

In [ ]:
delete_network_by_name(network_name)

#### Release Leases

In [ ]:
delete_lease_by_name(lease_name_network)
delete_lease_by_name(lease_name_servers)
delete_lease_by_name(lease_name_stitch)